# Modelado

En este notebook hacemos todo el proceso de fine-tuning de los 3 modelos diferente y evaluamos su performance.

# 0. Librerías 

In [1]:
1+1

2

# 1. Datos preprocesados

In [2]:
1+1

2

# 2. Tokenization

# 3. TensorDataset

In [3]:
1+1

2

# 4. DataLoaders

In [4]:
1+1

2

# 5. Función de Pérdida

In [5]:
1+1

2

# 6. Entrenamiento de BERT

In [8]:
1+1

2

# 7. Evaluación de BERT

In [9]:
1+1

2